In [ ]:
import numpy as np
import pandas as pd


In [ ]:
#read NFA-null from csv
inputFile="C:/Users/Admin/Desktop/FA CSV input files/NFANull.csv"
NFAnull = pd.read_csv(inputFile, index_col="states",dtype='str') 
NFAnull.index = NFAnull.index.map(str)

# convert data to appropriate type
NFAnull.isInitial = NFAnull["isInitial"].map({'FALSE':False, 'TRUE':True}) # bool
NFAnull.isFinal = NFAnull["isFinal"].map({'FALSE':False, 'TRUE':True}) # bool
# convert string transitions to sets.
for s in [x for x in list(NFAnull.columns) if x.find("input")!=-1]: 
    NFAnull[s]=[set(x.split(",")) if x==x else set() for x in NFAnull[s]]
print(NFAnull)

# find intial state, final state and symbols
q0= NFAnull.loc[NFAnull["isInitial"]==True].index.values[0]
print("inital State of NFAnull=",q0)
finalStates=set(NFAnull.loc[NFAnull["isFinal"]==True].index.values.tolist())
print("final states: ", finalStates)
symbols= [x for x in list(NFAnull.columns) if x.find("input")!=-1 and x!='input_null']
print("symbols: ", symbols) 



In [ ]:
# Null closure function of set state/ or single state
def nullclosure(states, NFANull):
    nullClosure=set()
    # add given set of single state to queue
    Queue= list(states)
    while len(Queue)>0:        
        state=Queue.pop() # take one state out of queue
        nullClosure.add(state) # add it to null closure
        # if their is some statex which can be reached from state add statex to queue.
        if(NFANull.loc[str(state), "input_null"]==NFANull.loc[str(state), "input_null"]):
                Queue= Queue+ list(NFANull.loc[str(state), "input_null"])
    return nullClosure  

In [ ]:
# evaluate null closure of all states in NFAnull,add to one column 
NFAnull["NClsr"] = [nullclosure(x,NFAnull) for x in NFAnull.index]

In [ ]:
# Create and NFA
NFA=pd.DataFrame(index=NFAnull.index ,columns= symbols+ ["isInitial", "isFinal"])

#set transitions for each symbol
for s in symbols:
    NFA[s]= NFAnull["NClsr"] # first null closure
    # transitions
    NFA[s]= [set.union(*[NFAnull.loc[y,s] for y in x]) for x in NFA[s]]
    # 2nd null closure
    NFA[s]=[ nullclosure(x, NFAnull) for x in NFA[s] ]

In [ ]:
# set final and intials state
NFA.loc[q0, "isInitial"]= True
# set final state
NFA.loc[list(finalStates), "isFinal"]= True
if len(NFAnull.loc[q0,"NClsr"].intersection(finalStates))>0:
    NFA.loc[q0, "isFinal"]=True

In [ ]:
print("Given NFA Null \n")
print(NFAnull)
print("\n\nCorresponding  NFA  \n")
print(NFA)